data source: http://help.sentiment140.com/for-students

learn bert NLP course https://www.udemy.com/course/bert-nlp-algorithm/learn/lecture/17347024#overview

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random


from google.colab import drive

In [ ]:
  !pip install bert-for-tf2

     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30537 sha256=45287a7599e8ec07d6edd7816c056afb12e95271bb2951c25cb3dec60fb9bdef
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7303 sha256=e53306f1e35643b19e49258e84ca7143c57868e9627cc7ea38208f19b1223300
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19474 sha256=a1c6dfbea0a524e88d3e61b526ed0d793441af07261fa2ed1c694e9268a48fc2
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 21.1MB/s 


In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
print(tf.version.VERSION)

2.4.0


In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


## Read data and load to dataframe

In [ ]:
columns = ["sentiment", "id", "date", "query", "user", "text"]

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/BERT/sentiment_data/train.csv",
                   header=None, names=columns, engine="python", encoding="latin1")

In [ ]:
data.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.drop(["id", "date", "query", "user"], inplace=True, axis=1)

In [ ]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data[data["sentiment"]!=0].head()

,sentiment,text
800000,4,I LOVE @Health4UandPets u guys r the best!!
800001,4,im meeting up with one of my besties tonight! ...
800002,4,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,4,Being sick can be really cheap when it hurts t...
800004,4,@LovesBrooklyn2 he has that effect on everyone


## DATA cleaning

In [ ]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, "lxml").get_text()
  tweet = re.sub(r'@[A-Za-z0-9]+', ' ', tweet)
  tweet = re.sub(r'https?://[A-Za-z0-9./]+', ' ', tweet)
  # tweet = re.sub(r'[a-zA-Z\.?!\'"]+', ' ', tweet)
  tweet = re.sub(r'\s+', ' ', tweet)
  tweet = re.sub(r'^\s+', '', tweet)
  return tweet

In [ ]:
data_sample = data.head().copy()

In [ ]:
data_sample["clean_text"] = data_sample["text"].apply(clean_tweet)

In [ ]:
data_sample.head()

,sentiment,text,clean_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- Awww, that's a bummer. You shoulda got David..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to sa...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am ..."


In [ ]:
data["clean_text"] = data["text"].apply(clean_tweet)

In [ ]:
data.head()

,sentiment,text,clean_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- Awww, that's a bummer. You shoulda got David..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to sa...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am ..."


In [ ]:
data["clean_text"] = data["text"].apply(clean_tweet)

In [ ]:
data.loc[(data.sentiment==4), "sentiment"] = 1

In [ ]:
data[data["sentiment"]!=0].head()

,sentiment,text,clean_text
800000,1,I LOVE @Health4UandPets u guys r the best!!,I LOVE u guys r the best!!
800001,1,im meeting up with one of my besties tonight! ...,im meeting up with one of my besties tonight! ...
800002,1,"@DaRealSunisaKim Thanks for the Twitter add, S...","Thanks for the Twitter add, Sunisa! I got to m..."
800003,1,Being sick can be really cheap when it hurts t...,Being sick can be really cheap when it hurts t...
800004,1,@LovesBrooklyn2 he has that effect on everyone,he has that effect on everyone


## Tokenisation

In [ ]:
FullTokeniser = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() 
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokeniser(vocab_file, do_lower_case)

In [ ]:
tokenizer.tokenize("My dog loves strawberries.")

['my', 'dog', 'loves', 'straw', '##berries', '.']

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("My dog loves strawberries."))

[2026, 3899, 7459, 13137, 20968, 1012]

In [ ]:
  def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [ ]:
data["data_inputs"] = data["clean_text"].apply(encode_sentence)

In [ ]:
data.head()

,sentiment,text,clean_text,data_inputs
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- Awww, that's a bummer. You shoulda got David...","[1011, 22091, 2860, 2860, 1010, 2008, 1005, 10..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...,"[2003, 6314, 2008, 2002, 2064, 1005, 1056, 106..."
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to sa...,"[1045, 11529, 2094, 2116, 2335, 2005, 1996, 36..."
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[2026, 2878, 2303, 5683, 2009, 11714, 1998, 20..."
4,0,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am ...","[2053, 1010, 2009, 1005, 1055, 2025, 2022, 327..."


## dataset creation

We will create padded batches (so we pad sentences for each batch independently). This way we add the minimum number of padded tokens possible. For that we sort sentences by length, add padded_batches and then shuffle

In [ ]:
data["len"] = data["data_inputs"].str.len()

In [ ]:
data.head()

,sentiment,text,clean_text,data_inputs,len
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- Awww, that's a bummer. You shoulda got David...","[1011, 22091, 2860, 2860, 1010, 2008, 1005, 10...",27
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...,"[2003, 6314, 2008, 2002, 2064, 1005, 1056, 106...",29
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to sa...,"[1045, 11529, 2094, 2116, 2335, 2005, 1996, 36...",20
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[2026, 2878, 2303, 5683, 2009, 11714, 1998, 20...",11
4,0,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am ...","[2053, 1010, 2009, 1005, 1055, 2025, 2022, 327...",33


In [ ]:
data["data_with_len"] = data[["data_inputs", "sentiment"]].apply(lambda a: tuple(a), axis=1)

In [ ]:
data.head()

,sentiment,text,clean_text,data_inputs,len,data_with_len
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- Awww, that's a bummer. You shoulda got David...","[1011, 22091, 2860, 2860, 1010, 2008, 1005, 10...",27,"([1011, 22091, 2860, 2860, 1010, 2008, 1005, 1..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...,"[2003, 6314, 2008, 2002, 2064, 1005, 1056, 106...",29,"([2003, 6314, 2008, 2002, 2064, 1005, 1056, 10..."
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to sa...,"[1045, 11529, 2094, 2116, 2335, 2005, 1996, 36...",20,"([1045, 11529, 2094, 2116, 2335, 2005, 1996, 3..."
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[2026, 2878, 2303, 5683, 2009, 11714, 1998, 20...",11,"([2026, 2878, 2303, 5683, 2009, 11714, 1998, 2..."
4,0,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am ...","[2053, 1010, 2009, 1005, 1055, 2025, 2022, 327...",33,"([2053, 1010, 2009, 1005, 1055, 2025, 2022, 32..."


In [ ]:
# do a random shuffling
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
data.head()

,sentiment,text,clean_text,data_inputs,len,data_with_len
0,0,@sarah_tov I need to get my bellybutton repier...,_tov I need to get my bellybutton repierced Yo...,"[1035, 2000, 2615, 1045, 2342, 2000, 2131, 202...",33,"([1035, 2000, 2615, 1045, 2342, 2000, 2131, 20..."
1,0,going to the doctor,going to the doctor,"[2183, 2000, 1996, 3460]",4,"([2183, 2000, 1996, 3460], 0)"
2,1,Up on a early AM. Running around getting somet...,Up on a early AM. Running around getting somet...,"[2039, 2006, 1037, 2220, 2572, 1012, 2770, 210...",32,"([2039, 2006, 1037, 2220, 2572, 1012, 2770, 21..."
3,1,@Felysha And congrats on being called the prop...,And congrats on being called the proper pronou...,"[1998, 26478, 8609, 2015, 2006, 2108, 2170, 19...",21,"([1998, 26478, 8609, 2015, 2006, 2108, 2170, 1..."
4,1,"@primraiyregin Aww, thank you!","Aww, thank you!","[22091, 2860, 1010, 4067, 2017, 999]",6,"([22091, 2860, 1010, 4067, 2017, 999], 1)"


In [ ]:
# sort the values
data.sort_values(by=["len"], inplace=True)

In [ ]:
data.head()

,sentiment,text,clean_text,data_inputs,len,data_with_len
591426,1,@jaffajaked,,[],0,"([], 1)"
322836,0,@shaundiviney,,[],0,"([], 0)"
1216187,1,@deanobean9,,[],0,"([], 1)"
1050903,1,@GtotheJan,,[],0,"([], 1)"
896000,1,@clairehearty,,[],0,"([], 1)"


In [ ]:
# we dont want the tweets that dont have data
data = data[data["len"]>2]

In [ ]:
data.head()

,sentiment,text,clean_text,data_inputs,len,data_with_len
1417891,0,No dodgems,No dodgems,"[2053, 11898, 5244]",3,"([2053, 11898, 5244], 0)"
517302,1,@chandlah have fun.,have fun.,"[2031, 4569, 1012]",3,"([2031, 4569, 1012], 1)"
357698,1,@keyvan take care pal,take care pal,"[2202, 2729, 14412]",3,"([2202, 2729, 14412], 1)"
491708,0,@Uniteu071 not too cool,not too cool,"[2025, 2205, 4658]",3,"([2025, 2205, 4658], 0)"
1419482,1,@ElizaJericho thanks for following,thanks for following,"[4283, 2005, 2206]",3,"([4283, 2005, 2206], 1)"


In [ ]:
sorted_all = data.data_with_len.values

In [ ]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
next(iter(all_dataset))

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 2053, 11898,  5244], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>)

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
NB_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10  
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

## Model building

In [ ]:
class DCNN(tf.keras.Model):
  def __init__(self, vocab_size, emb_dim=128, nb_filters=50,
               FFN_units=512, nb_classes=2, dropout_rate=0.1,
               training=False, name="dcnn"):
    super(DCNN, self).__init__(name=name)
    self.embedding = layers.Embedding(vocab_size, emb_dim)
    self.bigram = layers.Conv1D(filters=nb_filters, kernel_size=2,
                                padding="valid", activation="relu")
    self.trigram = layers.Conv1D(filters=nb_filters, kernel_size=3,
                                 padding="valid", activation="relu")
    self.fourgram = layers.Conv1D(filters=nb_filters, kernel_size=4,
                                  padding="valid", activation="relu")
    self.pool = layers.GlobalMaxPooling1D()
    self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
    self.dropout = layers.Dropout(rate=dropout_rate)
    if nb_classes == 2:
      self.last_dense = layers.Dense(units=1, activation="sigmoid")
    else:
      self.last_dense = layers.Dense(units=nb_classes, activation="softmax")

  def call(self, inputs, training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool(x_3)
    merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3*nb_filters)
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output

## training ophase

In [ ]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTER = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
dcnn = DCNN(vocab_size=VOCAB_SIZE, emb_dim=EMB_DIM, nb_filters=NB_FILTER,
            FFN_units=FFN_UNITS, nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
  dcnn.compile(loss="binary_crossentropy",
               optimizer="adam", metrics=["accuracy"])
else:
  dcnn.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
               metrics=["sparse_categorical_accuracy"])

In [ ]:
checkpoint_path = "/content/drive/MyDrive/BERT/ckpt_bert_tok"

ckpt = tf.train.Checkpoint(dcnn=dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("latest checkpoint restored!")

latest checkpoint restored!


In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epock, logs=None):
    ckpt_manager.save()
    print("checkpoint saved at {}".format(checkpoint_path))

In [ ]:
dcnn.fit(train_dataset, epochs=NB_EPOCHS, callbacks=[MyCustomCallback()])

Epoch 1/5
44318/44318 [==============================] - 2451s 55ms/step - loss: 0.3098 - accuracy: 0.8698
checkpoint saved at /content/drive/MyDrive/BERT/ckpt_bert_tok
Epoch 2/5
17726/44318 [==========>...................] - ETA: 24:37 - loss: 0.2748 - accuracy: 0.8874Buffered data was truncated after reaching the output size limit.

## model evaluation

In [ ]:
results = dcnn.evaluate(test_dataset)
print(results)

4924/4924 [==============================] - 26s 5ms/step - loss: 0.4632 - accuracy: 0.7652
[0.46315643191337585, 0.76524418592453]


In [ ]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)
  inputs = tf.expand_dims(tokens, 0)

  output = dcnn(inputs, training=False)
  
  sentiment = math.floor(output * 2)

  if sentiment == 0:
    p = "positive"
  else:
    p = "negative"
  print("output of the model: {}".format(output))
  print("predicted sentiment: {}".format(p))

In [ ]:
get_prediction("This movie was pretty interesting.")

output of the model: [[0.99120367]]
predicted sentiment: negative


In [ ]:
get_prediction("This movie was shit")

output of the model: [[0.05238095]]
predicted sentiment: positive
